In [1]:
import pickle
import numpy as np

In [2]:
# load the pkl
with open("pca_features.pkl", "rb") as file:
    track_ids = pickle.load(file)
    features_pca = pickle.load(file)

In [3]:
print(len(track_ids))
print(features_pca.shape)

11883
(237660, 154)


In [4]:
track_ids

['7pse475uICmWRY5hEkvPvI',
 '6AoYWCzsDls6c61Zp5qTuU',
 '6QZtmgaUkxWk3Rlq3Upll6',
 '1jBmW1fRXgLhy1XwE0SJTN',
 '4GYWNXDrG5wWOJteicSy7b',
 '0r0UDwngm74U5ccMQ9gMBM',
 '3Ot04s80MZBhiYP2wHo0is',
 '3EYTVekyY04eqdoi2iGlgJ',
 '4NQQ5fK6qOEMsJNlhjUkIg',
 '2dLWb2s1QtgpzAGOaBVa0h',
 '4UipF3IyxsyStTstiwGHhV',
 '1mnqraQ8oV8MX92rdOFLWW',
 '4QtiVmuA88tPQiCOHZuQ5b',
 '4dGLMiOBMUD74iiAGQ17cK',
 '41SwdQIX8Hy2u6fuEDgvWr',
 '6Ti0coAmnSsv3Wx09RhoY6',
 '78L2RUFISfqkmZMTCLYQJC',
 '28XvT0F0E4sVamI6JRVnmU',
 '2j7mcsEz4n5XxMMqqjjB6p',
 '5fIq2ovEhDIyV5Hx1epC4c',
 '5o7ZDvfOry1MNhNLBOJulo',
 '3HYXqOAm2Q8b2OyYqzv8d4',
 '4ESkG5iVSKtUANJK0NPz1w',
 '7KbF6AdprOXEEHlsq11Z6d',
 '6JVn2NEZ3JQLaHX5qG2Yho',
 '6k5qYxiFt76meFmKRDtZEz',
 '5O3nMjk8xL7n4TIh0tym4i',
 '50oMV9ChVZchW4zAcsKigA',
 '5AtVmp8JU2aar7PUKx6pdh',
 '7jHJ2MzIKO1kIMxGamKRKn',
 '15DwFznkBJir7AK9PyMyRR',
 '5UVBumEwdUnzvqxrXOYLFA',
 '0qgrrDnUUhyxpxbBznUnzg',
 '13aKGvTil3UUw1yk0pA9fG',
 '1HlGblweLq0eAwnb0NY1EQ',
 '6Nx1opJm0kavd1WWMpnlho',
 '0Mft55GDMuD2rFnrSeFn3U',
 

In [5]:
features_pca_1000=features_pca
track_ids_1000=track_ids

In [6]:
features_pca_1000[:20].shape

(20, 154)

In [184]:
import numpy as np
import heapq
from collections import Counter

def knn_lineal_pca(cancionesc,track_ids,consultas, top_k):
    all_neighbors=[]
    
    for consulta in consultas:
        nearest_neighbors = [(-float('inf'), -1)] * 20 #20 caracteristicas
        for i, embedding in enumerate(cancionesc):
            dist = -np.linalg.norm(consulta - embedding)#consigo mi distancia euclidiana invertida para usar como max-heap
            if dist > nearest_neighbors[0][0]:
                heapq.heappop(nearest_neighbors)
                heapq.heappush(nearest_neighbors, (dist, i))
        nearest_neighbors = [i for dit, i in nearest_neighbors]
        all_neighbors.extend(nearest_neighbors)

    song_indices = [neighbor // 20 for neighbor in all_neighbors]
    track_id_votes = [track_ids[song_idx] for song_idx in song_indices]

    song_counts = Counter(track_id_votes)
    top_songs = song_counts.most_common(top_k)  

    return top_songs


indicecancion = 0 
query = features_pca[indicecancion * 20 : (indicecancion + 1) * 20]  # consulta
top_k =30  #cuantos k vecinos cercanos quiero

resultado = knn_lineal_pca(features_pca, track_ids, query, top_k)

# Mostrar resultados
for i, (song_id, votes) in enumerate(resultado, 1):
    print(f"Song {i}: '{song_id}', Votes: {votes}")


Song 1: '7pse475uICmWRY5hEkvPvI', Votes: 23
Song 2: '5CwOUooch74h0XarhDfAQK', Votes: 10
Song 3: '3bZCS8ThTAxMJZavYWOY1z', Votes: 8
Song 4: '1U2xFfjK1QUuicENnW0iwv', Votes: 4
Song 5: '3OiEY2VLzrTyCoU8q2SQpe', Votes: 4
Song 6: '6MGryNr7aENIEfPUV1cHyg', Votes: 4
Song 7: '1oy6EH41CdAido7rIuuFzY', Votes: 4
Song 8: '2vPZ4Lklyu75zBR3SgbFNI', Votes: 4
Song 9: '1wo3UYTeizJHkwYIuLuBPF', Votes: 3
Song 10: '2HuKPBTjdwDz4AgyScuMDe', Votes: 3
Song 11: '760xFXFj5Fksa3jawSCn5g', Votes: 3
Song 12: '0uLdF7n18IFclndsRWaqx0', Votes: 3
Song 13: '1Krtfh550kyhfWPcRp6SnD', Votes: 3
Song 14: '4CBgcRYGyWjGGaSFuPJChO', Votes: 3
Song 15: '79MMMdYL00iwKVHBSAUkLY', Votes: 3
Song 16: '0WrrIkbhC2H4WRqxqY1Xot', Votes: 3
Song 17: '672N8DGGTOLCOgWe0koX5g', Votes: 3
Song 18: '2upFJKo7EvWdMx5732mhuc', Votes: 2
Song 19: '2aICaLTkejrkgsLeCsbQY6', Votes: 2
Song 20: '5HLes17mwCwKt81mi0Tk35', Votes: 2
Song 21: '1zcJ1eMjRONHitpcPx0c68', Votes: 2
Song 22: '0NrWlujfMeGgRlWRJi00zL', Votes: 2
Song 23: '3s2C5xTanm4Kk3Ed2Rb2mo', Vote

busqueda por rango

In [ ]:
import numpy as np
import heapq
from collections import Counter

def knn_busquedarango_pca(cancionesc, track_ids, consultas,radio):
    all_neighbors = []
    distancias = [] 

    for consulta in consultas:
        nearest_neighbors = [(-float('inf'), -1)] * 20  # 20 características
        for i, embedding in enumerate(cancionesc):
            dist = -np.linalg.norm(consulta - embedding)  # Distancia euclidiana invertida para max-heap
            if dist > nearest_neighbors[0][0]:
                heapq.heappop(nearest_neighbors)
                heapq.heappush(nearest_neighbors, (dist, i))
        nearest_neighbors = [(dit, i) for dit, i in nearest_neighbors]
        all_neighbors.extend([i for dit, i in nearest_neighbors])
        distancias.extend([-dit for dit, i in nearest_neighbors])  

    song_indices = [neighbor // 20 for neighbor in all_neighbors]
    track_id_votes = [track_ids[song_idx] for song_idx in song_indices] 
    song_counts = Counter(track_id_votes)

    track_id_distancias = {}
    for idx, track_id in enumerate(track_id_votes):
        if track_id not in track_id_distancias:
            track_id_distancias[track_id] = []
        track_id_distancias[track_id].append(distancias[idx])
    distancias_canciones = [(song_id, votes, np.mean(track_id_distancias[song_id])) for song_id, votes in song_counts.most_common()]
    top_songs = []
    for i, (song_id, votes, avg_dist) in enumerate(distancias_canciones):
        if i < 10 or avg_dist <= radio:
            top_songs.append((song_id, votes))

    return top_songs


# Ejemplo de Uso
indicecancion = 0
query = features_pca[indicecancion * 20 : (indicecancion + 1) * 20]
radio = 0.3  

resultado = knn_busquedarango_pca(features_pca, track_ids, query,radio)

# Mostrar resultados
print("Numero de canciones obtenidas: ", len(resultado))
for i, (song_id, votes) in enumerate(resultado, 1):
    print(f"Song {i}: '{song_id}', Votes: {votes}")


Numero de canciones obtenidas:  108
Song 1: '7pse475uICmWRY5hEkvPvI', Votes: 23
Song 2: '5CwOUooch74h0XarhDfAQK', Votes: 10
Song 3: '3bZCS8ThTAxMJZavYWOY1z', Votes: 8
Song 4: '1U2xFfjK1QUuicENnW0iwv', Votes: 4
Song 5: '3OiEY2VLzrTyCoU8q2SQpe', Votes: 4
Song 6: '6MGryNr7aENIEfPUV1cHyg', Votes: 4
Song 7: '1oy6EH41CdAido7rIuuFzY', Votes: 4
Song 8: '2vPZ4Lklyu75zBR3SgbFNI', Votes: 4
Song 9: '1wo3UYTeizJHkwYIuLuBPF', Votes: 3
Song 10: '2HuKPBTjdwDz4AgyScuMDe', Votes: 3
Song 11: '4CBgcRYGyWjGGaSFuPJChO', Votes: 3
Song 12: '79MMMdYL00iwKVHBSAUkLY', Votes: 3
Song 13: '0WrrIkbhC2H4WRqxqY1Xot', Votes: 3
Song 14: '672N8DGGTOLCOgWe0koX5g', Votes: 3
Song 15: '7LVHVU3tWfcxj5aiPFEW4Q', Votes: 2
Song 16: '5zsgyaahJyDW91qyvzSG3Y', Votes: 2
Song 17: '1Yf6xbi1dJVdiKoWwxL1Ti', Votes: 2
Song 18: '59NImYypcsmmw1zeaP5e7z', Votes: 2
Song 19: '69Op4XR486hl5LSiBtPHmp', Votes: 2
Song 20: '3jnTBMdckWNzC7PJV96kQ1', Votes: 2
Song 21: '2YIkyCfSVOoMUfN22gqttg', Votes: 2
Song 22: '0s1PewZjIU8DJqxktjeZaX', Votes: 2
Son

In [182]:
radio = 0.4 

resultado = knn_busquedarango_pca(features_pca, track_ids, query,radio)

# Mostrar resultados
print("Numero de canciones obtenidas: ", len(resultado))
for i, (song_id, votes) in enumerate(resultado, 1):
    print(f"Song {i}: '{song_id}', Votes: {votes}")

Numero de canciones obtenidas:  232
Song 1: '7pse475uICmWRY5hEkvPvI', Votes: 23
Song 2: '5CwOUooch74h0XarhDfAQK', Votes: 10
Song 3: '3bZCS8ThTAxMJZavYWOY1z', Votes: 8
Song 4: '1U2xFfjK1QUuicENnW0iwv', Votes: 4
Song 5: '3OiEY2VLzrTyCoU8q2SQpe', Votes: 4
Song 6: '6MGryNr7aENIEfPUV1cHyg', Votes: 4
Song 7: '1oy6EH41CdAido7rIuuFzY', Votes: 4
Song 8: '2vPZ4Lklyu75zBR3SgbFNI', Votes: 4
Song 9: '1wo3UYTeizJHkwYIuLuBPF', Votes: 3
Song 10: '2HuKPBTjdwDz4AgyScuMDe', Votes: 3
Song 11: '0uLdF7n18IFclndsRWaqx0', Votes: 3
Song 12: '1Krtfh550kyhfWPcRp6SnD', Votes: 3
Song 13: '4CBgcRYGyWjGGaSFuPJChO', Votes: 3
Song 14: '79MMMdYL00iwKVHBSAUkLY', Votes: 3
Song 15: '0WrrIkbhC2H4WRqxqY1Xot', Votes: 3
Song 16: '672N8DGGTOLCOgWe0koX5g', Votes: 3
Song 17: '4XwLxC47UNg3EByUt5RTIG', Votes: 2
Song 18: '0coMLoVcagZPGF5zxc5RF8', Votes: 2
Song 19: '3h9uL1GB5GU0TdNPrFHluJ', Votes: 2
Song 20: '0rZTJmLQx7IquBctp98d0z', Votes: 2
Song 21: '4dEZbrokJ8lgNyrdL9ZTe8', Votes: 2
Song 22: '0GQjTATSapMJFHoKXiqprG', Votes: 2
Son

In [183]:
radio = 0.5

resultado = knn_busquedarango_pca(features_pca, track_ids, query,radio)

# Mostrar resultados
print("Numero de canciones obtenidas: ", len(resultado))
for i, (song_id, votes) in enumerate(resultado, 1):
    print(f"Song {i}: '{song_id}', Votes: {votes}")

Numero de canciones obtenidas:  247
Song 1: '7pse475uICmWRY5hEkvPvI', Votes: 23
Song 2: '5CwOUooch74h0XarhDfAQK', Votes: 10
Song 3: '3bZCS8ThTAxMJZavYWOY1z', Votes: 8
Song 4: '1U2xFfjK1QUuicENnW0iwv', Votes: 4
Song 5: '3OiEY2VLzrTyCoU8q2SQpe', Votes: 4
Song 6: '6MGryNr7aENIEfPUV1cHyg', Votes: 4
Song 7: '1oy6EH41CdAido7rIuuFzY', Votes: 4
Song 8: '2vPZ4Lklyu75zBR3SgbFNI', Votes: 4
Song 9: '1wo3UYTeizJHkwYIuLuBPF', Votes: 3
Song 10: '2HuKPBTjdwDz4AgyScuMDe', Votes: 3
Song 11: '760xFXFj5Fksa3jawSCn5g', Votes: 3
Song 12: '0uLdF7n18IFclndsRWaqx0', Votes: 3
Song 13: '1Krtfh550kyhfWPcRp6SnD', Votes: 3
Song 14: '4CBgcRYGyWjGGaSFuPJChO', Votes: 3
Song 15: '79MMMdYL00iwKVHBSAUkLY', Votes: 3
Song 16: '0WrrIkbhC2H4WRqxqY1Xot', Votes: 3
Song 17: '672N8DGGTOLCOgWe0koX5g', Votes: 3
Song 18: '4XwLxC47UNg3EByUt5RTIG', Votes: 2
Song 19: '7v0mtl6oInUtHOmTk2b0gC', Votes: 2
Song 20: '0coMLoVcagZPGF5zxc5RF8', Votes: 2
Song 21: '3h9uL1GB5GU0TdNPrFHluJ', Votes: 2
Song 22: '0rZTJmLQx7IquBctp98d0z', Votes: 2
Son

Testando la similitud 

In [20]:
import pandas as pd
df = pd.read_csv('spotify_songs.csv',sep=',')

# Mostrar las primeras filas del DataFrame
df

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.011700,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.009940,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.000000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.000000,0.1380,0.240,130.018,193160,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18449,7zXzWCVuz7whIuYZyEAlxt,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,1l4aoukbPgi5u2OaE2R4Zj,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,...,-5.778,0,0.0878,0.00555,0.000000,0.3350,0.211,128.012,208656,en
18450,7zycSpvjDcqh6YT1FEl2kY,Anaconda,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon...",49,5qs8T6ZHSrnllnOuUk6muC,The Pinkprint (Deluxe Edition),2014-12-15,10er Playlist,1kEczIkZH8IgaWT2BiApxZ,...,-6.224,1,0.1800,0.06730,0.000006,0.2140,0.647,129.990,260240,en
18451,7zye9v6B785eFWEFYs13C2,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,1xdgLmTFMSyJyI5DJOOX7T,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,5EMARioe9z9eKOeWIAC2JW,...,-6.457,0,0.0270,0.71500,0.000428,0.1150,0.657,142.218,191205,en
18452,7zyLObYw4QUKQDyZOb4J0Y,I'll Do 4 U (Re-Recorded / Remastered),Father MC,(Would you do for me) Sweetheart (Would you do...,36,14HYMxFhpgDIr9cci1u0kt,I'll Do 4 U (Re-Recorded / Remastered),2010-10-01,New Jack Swing/ R&B Hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,...,-4.920,0,0.0633,0.14300,0.000000,0.0720,0.810,109.536,223890,en


In [ ]:
df.iloc[[13,14,95,471,568]]


In [90]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('spotify_songs.csv', sep=',')

filas_seleccionadas = df.iloc[[14, 15, 96, 472, 569]]

caracteristicas_acusticas = ['danceability', 'energy', 'loudness', 'tempo', 'duration_ms', 'valence']

similaridad = cosine_similarity(filas_seleccionadas[caracteristicas_acusticas])
similaridad_df = pd.DataFrame(similaridad, index=filas_seleccionadas.index, columns=filas_seleccionadas.index)

print(similaridad_df)



     14   15   96   472  569
14   1.0  1.0  1.0  1.0  1.0
15   1.0  1.0  1.0  1.0  1.0
96   1.0  1.0  1.0  1.0  1.0
472  1.0  1.0  1.0  1.0  1.0
569  1.0  1.0  1.0  1.0  1.0
